In [57]:
num_classes = 1+10

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# print(model)
# move model to the right device
model.to(device)
model.load_state_dict(torch.load('D:/KJE_Airiss/cow_data_for_AI/all_dataset_rm_bg_cow_deform_no_rotate_with_json3(500x500)/save_model/learning_rate=0.005/epoch_new_1.pt')
)


<All keys matched successfully>

In [1]:
import cv2
import numpy as np
mask_colors=[[1,1,0],[1,0,1],[0,0,1],[0,1,0],[1,0,0],[0,1,1],[0.5,0.5,0],[0.7,0.3,0.5],[1,0.5,0.1],[0.3,0.5,0.8]]
import matplotlib.pyplot as plt


# root='C:/Users/AiRISS/container_sinsundae_resize/'
def coordinate_segm(each_segm):
            return [((each_segm[0][i],each_segm[0][i+1])) for i in range(0,len(each_segm[0]),2)]
import torch
import numpy
from PIL import Image
class CowDataset(torch.utils.data.Dataset):
    def __init__(self, root,imgs, transforms,pc_train_python):
        
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고, 정렬하여
        # 이미지와 분할 마스크 정렬을 확인합니다
#         self.imgs = list(sorted(os.listdir(os.path.join(root))))
        self.imgs = imgs
#         print(self.imgs)
        idx1=0
        annot_image_id=[]
        for ii in pc_train_python['annotations']:

            tmp=pc_train_python['annotations'][idx1]
            idx1=idx1+1
            annot_image_id.append(tmp['image_id'])  
            
        self.annot_image_id=annot_image_id
        self.pc_train_python  =pc_train_python  

        

    ###########################################################################
    def coordinate_segm(each_segm):
            return [((each_segm[0][i],each_segm[0][i+1])) for i in range(0,len(each_segm[0]),2)]

   

    def __getitem__(self, idx):
        
        # 이미지와 마스크를 읽어옵니다
#         self.root = root
# #         self.transforms = transforms
#         # 모든 이미지 파일들을 읽고, 정렬하여
#         # 이미지와 분할 마스크 정렬을 확인합니다
#         self.imgs = list(sorted(os.listdir(os.path.join(root))))
        
        pc_train_python    = self.pc_train_python           
        img_path = os.path.join(self.root, self.imgs[idx])
#         print(img_path)
        #         mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        #         img = cv2.imread('train/{}.png'.format(self.all_img_id[idx-1]),cv2.IMREAD_COLOR)
#         img2 = imread(img_path)
#         plt.figure
#         plt.imshow(img2)
        img = Image.open(img_path)

        # 분할 마스크는 RGB로 변환하지 않음을 유의하세요
        # 왜냐하면 각 색상은 다른 인스턴스에 해당하며, 0은 배경에 해당합니다
        
        for img_id in range(len(pc_train_python['images'])):
            all_segm=[]
            all_categ=[]
            all_img_id=[]
            all_bbox=[]
            all_area=[]
            all_iscrowd=[]
            if pc_train_python['images'][img_id]['file_name']==self.imgs[idx]:
#         for img_id in range((3)): 
                
                img_id1=int(pc_train_python['images'][img_id]['id'])
#                 print(img_id1)
                idx_segm=[ i for i,img_id2 in enumerate(self.annot_image_id) if img_id2==img_id1]
#                 print(idx_segm)
                segm=[]
                categ=[]
                img_id_tmp=[]
                boxes1=[]
                area1=[]
                all_img_id_bbox=[]
                all_img_id_area=[] 
                iscrowd=[]
                for ii in idx_segm:
                #     print(ii)
                    segm.append(pc_train_python['annotations'][ii]["segmentation"])
                    categ.append(pc_train_python['annotations'][ii]['category_id'])
                    tmp_bbox=pc_train_python['annotations'][ii]['bbox']

                    tmp_bbox[2], tmp_bbox[3] = tmp_bbox[0]+tmp_bbox[2], tmp_bbox[3]+tmp_bbox[1]

                    boxes1.append(tmp_bbox)
                    area1.append(pc_train_python['annotations'][ii]['area'])
                    img_id_tmp.append(pc_train_python['annotations'][ii]['image_id'])
                    iscrowd.append(int(pc_train_python['annotations'][ii]['iscrowd']))

#                 all_segm.append(segm)
#                 all_categ.append(categ)
#                 all_bbox.append(boxes1)
#                 all_area.append(area1)
#                 all_img_id.append(img_id1)
#                 all_iscrowd.append(iscrowd)
#                 print(len(segm))
        
        self.all_segm=segm
        self.all_categ=categ    
        self.all_bbox=boxes1
        self.all_area=area1                      
        self.all_img_id=img_id1
        self.all_iscrowd=iscrowd 
####################################################################################################################################       
        segm=self.all_segm  # [[segment][image_idx][segment_idx]]
        categ=self.all_categ # [[category][image_idx][segment_idx]]

        boxes=self.all_bbox
#         print(boxes)
        labels=self.all_categ
        image_id=self.all_img_id
        area=self.all_area
        iscrowd=self.all_iscrowd
#         print(img)
        img2=numpy.asarray(img)

        h,w,c= img2.shape

        segm2=segm
        mask=[]
#         print('length segmentation=',len(segm2))
        for ii in range(len(segm2)):
            mask3=[]
#             print(ii)
            mask1 = np.zeros((h,w))
            for dd in range(len(segm2[ii])):
#             for dd in [0]:
#                 
                each_segm=   segm2[ii][dd]  
                coodinate1=coordinate_segm([each_segm])
                arr = np.array(coodinate1, np.int32)
                mask1 = cv2.fillPoly(mask1, [arr], (1,1))
#                 plt.figure()
#                 plt.imshow(mask1)
            mask3=mask1.tolist()

            mask.append(mask3)
        masks=mask




#         with open(self.imgs[idx]+'_masks_'+ '.pkl', 'wb') as f:
#             pickle.dump(masks, f)


#         print('box=',boxes)
#         print('label=',categ)
#         print('masks=',len(masks))

#         print('image_id2=',image_id)
#         print('area2=',area)
#         print('iscrowd2=',iscrowd)
        # 모든 것을 torch.Tensor 타입으로 변환합니다
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 객체 종류는 한 종류만 존재합니다(역자주: 예제에서는 사람만이 대상입니다)
#         labels = torch.as_tensor(labels2, dtype=torch.int64)

        labels = torch.as_tensor(categ, dtype=torch.int64)
        masks = torch.as_tensor( masks, dtype=torch.uint8)

        image_id = torch.as_tensor([image_id])
        area = torch.as_tensor( area)
        # 모든 인스턴스는 군중(crowd) 상태가 아님을 가정합니다
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)  
    
    
# root='C:/Users/AiRISS/container_sinsundae_resize/'
import torch
import numpy
from PIL import Image

class CowDataset_test(torch.utils.data.Dataset):
    def __init__(self, root,imgs, transforms,pc_train_python):
        
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고, 정렬하여
        # 이미지와 분할 마스크 정렬을 확인합니다
#         self.imgs = list(sorted(os.listdir(os.path.join(root))))
        self.imgs = imgs
#         print(self.imgs)
        idx1=0
        annot_image_id=[]
        for ii in pc_train_python['annotations']:

            tmp=pc_train_python['annotations'][idx1]
            idx1=idx1+1
            annot_image_id.append(tmp['image_id'])  
            
        self.annot_image_id=annot_image_id
        self.pc_train_python=pc_train_python
    

        

    ###########################################################################
    def coordinate_segm(each_segm):
            return [((each_segm[0][i],each_segm[0][i+1])) for i in range(0,len(each_segm[0]),2)]

   

    def __getitem__(self, idx):
        
        # 이미지와 마스크를 읽어옵니다
#         self.root = root
# #         self.transforms = transforms
#         # 모든 이미지 파일들을 읽고, 정렬하여
#         # 이미지와 분할 마스크 정렬을 확인합니다
#         self.imgs = list(sorted(os.listdir(os.path.join(root))))
        
        pc_train_python    = self.pc_train_python      
        img_path = os.path.join(self.root, self.imgs[idx])
#         print(img_path)
        #         mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        #         img = cv2.imread('train/{}.png'.format(self.all_img_id[idx-1]),cv2.IMREAD_COLOR)
#         img2 = imread(img_path)
#         plt.figure
#         plt.imshow(img2)
        img = Image.open(img_path)

        # 분할 마스크는 RGB로 변환하지 않음을 유의하세요
        # 왜냐하면 각 색상은 다른 인스턴스에 해당하며, 0은 배경에 해당합니다
        segm=[]
        categ=[]
        img_id_tmp=[]
        img_id1=[]
        boxes1=[]
        area1=[]
        all_img_id_bbox=[]
        all_img_id_area=[] 
        iscrowd=[]
        
#         print(len(pc_train_python['images']))
        for img_id in range(len(pc_train_python['images'])):
            all_segm=[]
            all_categ=[]
            all_img_id=[]
            all_bbox=[]
            all_area=[]
            all_iscrowd=[]
            if pc_train_python['images'][img_id]['file_name']==self.imgs[idx]:
#         for img_id in range((3)): 
                
                img_id1=int(pc_train_python['images'][img_id]['id'])
#                 print(img_id1)
                idx_segm=[ i for i,img_id2 in enumerate(self.annot_image_id) if img_id2==img_id1]
#                 print(idx_segm)
                segm=[]
                categ=[]
                img_id_tmp=[]
                boxes1=[]
                area1=[]
                all_img_id_bbox=[]
                all_img_id_area=[] 
                iscrowd=[]
                for ii in idx_segm:
                #     print(ii)
                    segm.append(pc_train_python['annotations'][ii]["segmentation"])
                    categ.append(pc_train_python['annotations'][ii]['category_id'])
                    tmp_bbox=pc_train_python['annotations'][ii]['bbox']

                    tmp_bbox[2], tmp_bbox[3] = tmp_bbox[0]+tmp_bbox[2], tmp_bbox[3]+tmp_bbox[1]

                    boxes1.append(tmp_bbox)
                    area1.append(pc_train_python['annotations'][ii]['area'])
                    img_id_tmp.append(pc_train_python['annotations'][ii]['image_id'])
                    iscrowd.append(int(pc_train_python['annotations'][ii]['iscrowd']))

#                 all_segm.append(segm)
#                 all_categ.append(categ)
#                 all_bbox.append(boxes1)
#                 all_area.append(area1)
#                 all_img_id.append(img_id1)
#                 all_iscrowd.append(iscrowd)
#                 print(len(segm))
        
        self.all_segm=segm
        self.all_categ=categ    
        self.all_bbox=boxes1
        self.all_area=area1                      
        self.all_img_id=img_id_tmp
        self.all_iscrowd=iscrowd 
####################################################################################################################################       
        segm=self.all_segm  # [[segment][image_idx][segment_idx]]
        categ=self.all_categ # [[category][image_idx][segment_idx]]

        boxes=self.all_bbox
#         print(boxes)
        labels=self.all_categ
        image_id=self.all_img_id
        area=self.all_area
        iscrowd=self.all_iscrowd
#         print(img)
        img2=numpy.asarray(img)

        h,w,c= img2.shape

        segm2=segm
        mask=[]
#         print('length segmentation=',len(segm2))
        for ii in range(len(segm2)):
            mask3=[]
#             print(ii)
            mask1 = np.zeros((h,w))
            for dd in range(len(segm2[ii])):
#             for dd in [0]:
#                 
                each_segm=   segm2[ii][dd]  
                coodinate1=coordinate_segm([each_segm])
                arr = np.array(coodinate1, np.int32)
                mask1 = cv2.fillPoly(mask1, [arr], (1,1))
#                 plt.figure()
#                 plt.imshow(mask1)
            mask3=mask1.tolist()

            mask.append(mask3)
        masks=mask




#         with open(self.imgs[idx]+'_masks_'+ '.pkl', 'wb') as f:
#             pickle.dump(masks, f)
        image_id=np.unique(image_id)

#         print('box=',boxes)
#         print('label=',categ)
#         print('masks=',len(masks))

#         print('image_id2=',image_id)
#         print('area2=',area)
#         print('iscrowd2=',iscrowd)
        # 모든 것을 torch.Tensor 타입으로 변환합니다
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 객체 종류는 한 종류만 존재합니다(역자주: 예제에서는 사람만이 대상입니다)
#         labels = torch.as_tensor(labels2, dtype=torch.int64)

        labels = torch.as_tensor(categ, dtype=torch.int64)
        masks = torch.as_tensor( masks, dtype=torch.uint8)

        image_id = torch.as_tensor([image_id])
        area = torch.as_tensor( area)
        # 모든 인스턴스는 군중(crowd) 상태가 아님을 가정합니다
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)  
    
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # COCO 에서 미리 학습된 인스턴스 분할 모델을 읽어옵니다
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # 분류를 위한 입력 특징 차원을 얻습니다
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 미리 학습된 헤더를 새로운 것으로 바꿉니다
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 마스크 분류기를 위한 입력 특징들의 차원을 얻습니다
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # 마스크 예측기를 새로운 것으로 바꿉니다
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

import os
os.chdir('C:/Users/AiRISS/Container_python_analysis')
from vision.references.detection.engine import train_one_epoch, evaluate
import vision.references.detection.utils as utils
import vision.references.detection.transforms as T

from torchvision import transforms as T
import transforms as T
os.chdir('C:/Users/AiRISS')
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # (역자주: 학습시 50% 확률로 학습 영상을 좌우 반전 변환합니다)
        
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


import json

# root='C:/Users/AiRISS/yolact-container_bg_boxing/data/container_sinsundae_resize_gray_scale'
# root='C:\Users\AiRISS\yolact-container_bg_boxing\data\container_sinsundae_resize_gray_scale'
# use our dataset and defined transformations


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 1+23

# # get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# # # print(model)
# # # move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

num_classes = 1+23

# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 40
import numpy as np

for epoch in range(num_epochs):
    ################################                 load dataset                 ########################################
    with open("D:/KJE_Airiss/cow_data_for_AI/all_dataset_rm_bg_cow_deform_no_rotate_with_json3_new_correct_23class(500x500)/cow_json_train.json","r") as pc_train_json:  # pc : poice
            pc_train_python = json.load(pc_train_json)
    imgs=[]    
    for kk in range(len(pc_train_python['images'])):
        imgs.append(pc_train_python['images'][kk]['file_name'])
    root='D:/KJE_Airiss/cow_data_for_AI/all_dataset_rm_bg_cow_deform_no_rotate_with_json3_new_correct_23class(500x500)/train'
    dataset = CowDataset(root, imgs,get_transform(train=False),pc_train_python)


    with open("D:/KJE_Airiss/cow_data_for_AI/all_dataset_rm_bg_cow_deform_no_rotate_with_json3_new_correct_23class(500x500)/cow_json_valid.json","r") as pc_train_json:  # pc : poice
                pc_valid_python = json.load(pc_train_json)
    imgs_vals=[]    
    for kk in range(len(pc_valid_python['images'])):
        imgs_vals.append(pc_valid_python['images'][kk]['file_name'])
    root='D:/KJE_Airiss/cow_data_for_AI/all_dataset_rm_bg_cow_deform_no_rotate_with_json3_new_correct_23class(500x500)/valid'
    dataset_test = CowDataset_test(root, imgs_vals,get_transform(train=False),pc_valid_python)

    # split the dataset in train and test set
    # torch.manual_seed(1)
    indices = torch.randperm(len(dataset)).tolist()

    # indices = np.linspace(0,len(dataset)-1,len(dataset))
    # indices=list(indices)
    # indices = [int(item) for item in indices]
    # print(indices)
    dataset = torch.utils.data.Subset(dataset, indices)
    # dataset = torch.utils.data.Subset(dataset,indices )
    indices = torch.randperm(len(dataset_test)).tolist()
    indices = np.linspace(0,len(dataset_test)-1,len(dataset_test))
    indices=list(indices)
    indices = [int(item) for item in indices]
    dataset_test = torch.utils.data.Subset(dataset_test, indices)

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=3, shuffle=True, num_workers=0,
        collate_fn=utils.collate_fn)  # 원래는 num_workers=4 인데 이렇게 하면 에러남.
    data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)  
    
    ##################################                    train                 #########################################
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    torch.save(model.state_dict(), 'D:/KJE_Airiss/cow_data_for_AI/all_dataset_rm_bg_cow_deform_no_rotate_with_json3_new_correct_23class(500x500)/save_model/epoch_all_cow'+str(epoch) +'.pt')
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test,device='cuda')
#     print(eval1.results1)
#     model.eval()
#     x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
#     predictions = model(x)   

C:\Users\AiRISS\AppData\Local\Temp\ipykernel_8732\3895507037.py:169: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  masks = torch.as_tensor( masks, dtype=torch.uint8)


Epoch: [0]  [   0/4130]  eta: 13:02:51  lr: 0.000010  loss: 6.0813 (6.0813)  loss_classifier: 3.2190 (3.2190)  loss_box_reg: 0.8293 (0.8293)  loss_mask: 1.3363 (1.3363)  loss_objectness: 0.6282 (0.6282)  loss_rpn_box_reg: 0.0686 (0.0686)  time: 11.3731  data: 4.1954  max mem: 2741


KeyboardInterrupt: 

In [ ]:
categories=[{'id': 1,
  'name': '가시근',
  'supercategory': 'mucle',
  'color': '040439',
  'metadata': 'korean_beef'},
 {'id': 2,
  'name': '반가시근',
  'supercategory': 'mucle',
  'color': 'ba0e79',
  'metadata': 'korean_beef'},
 {'id': 3,
  'name': '널판근',
  'supercategory': 'mucle',
  'color': 'a6c2d0',
  'metadata': 'korean_beef'},
 {'id': 4,
  'name': '마름모근',
  'supercategory': 'mucle',
  'color': '542bb5',
  'metadata': 'korean_beef'},
 {'id': 5,
  'name': '등세모근',
  'supercategory': 'mucle',
  'color': '85a757',
  'metadata': 'korean_beef'},
 {'id': 6,
  'name': '배쪽톱니근',
  'supercategory': 'mucle',
  'color': '3b0411',
  'metadata': 'korean_beef'},
 {'id': 7,
  'name': '인대',
  'supercategory': 'mucle',
  'color': '628b89',
  'metadata': 'korean_beef'},
 {'id': 8,
  'name': '지방',
  'supercategory': 'mucle',
  'color': '68ab5b',
  'metadata': 'korean_beef'},
 {'id': 9,
  'name': '기타덧살',
  'supercategory': 'mucle',
  'color': 'd0534a',
  'metadata': 'korean_beef'},
 {'id': 10,
  'name': '최장근',
  'supercategory': 'mucle',
  'color': '609899',
  'metadata': 'korean_beef'},
{'id': 11,
  'name': 'bone01',
  'supercategory': 'bone',
  'color': '040439',
  'metadata': 'korean_beef'},
 {'id': 12,
  'name': 'bone02',
  'supercategory': 'bone',
  'color': 'ba0e79',
  'metadata': 'korean_beef'},
 {'id': 13,
  'name': 'bone03',
  'supercategory': 'bone',
  'color': 'a6c2d0',
  'metadata': 'korean_beef'},
 {'id': 14,
  'name': 'bone04',
  'supercategory': 'bone',
  'color': '542bb5',
  'metadata': 'korean_beef'},
 {'id': 15,
  'name': 'bone05',
  'supercategory': 'bone',
  'color': '85a757',
  'metadata': 'korean_beef'},
 {'id': 16,
  'name': 'bone06',
  'supercategory': 'bone',
  'color': '3b0411',
  'metadata': 'korean_beef'},
 {'id': 17,
  'name': 'bone07',
  'supercategory': 'bone',
  'color': '628b89',
  'metadata': 'korean_beef'},
 {'id': 18,
  'name': 'bone08',
  'supercategory': 'bone',
  'color': '68ab5b',
  'metadata': 'korean_beef'},
 {'id': 19,
  'name': 'bone09',
  'supercategory': 'bone',
  'color': 'd0534a',
  'metadata': 'korean_beef'},
 {'id': 20,
  'name': 'bone10',
  'supercategory': 'bone',
  'color': '609899',
  'metadata': 'korean_beef'},
  {'id': 21,
  'name': 'bone11',
  'supercategory': 'bone',
  'color': '68ab5b',
  'metadata': 'korean_beef'},
 {'id': 22,
  'name': 'bone12',
  'supercategory': 'bone',
  'color': 'd0534a',
  'metadata': 'korean_beef'},
 {'id': 23,
  'name': 'bone13',
  'supercategory': 'bone',
  'color': '609899',
  'metadata': 'korean_beef'}]